In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Extração unitaria

In [2]:
website_url = requests.get('https://pt.wikipedia.org/wiki/S%C3%A3o_Paulo_(estado)').text
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

My_table = soup.find('table')

In [3]:
i =0
links = My_table.find(title="Fronteira")
texto = []
hrefs = []
while i == 0:
    if links.find_next('a').text != "Regiões geográficas intermediárias":
        links = links.find_next('a')
        texto.append(links.text)
        hrefs.append(links.get('href'))
    else:
        i=1
        
df = pd.DataFrame([texto, hrefs] ).T
df.columns = ['Estado_Vizinho', 'link_wikipedia']
df

,Estado_Vizinho,link_wikipedia
0,Paraná,/wiki/Paran%C3%A1
1,Mato Grosso do Sul,/wiki/Mato_Grosso_do_Sul
2,Minas Gerais,/wiki/Minas_Gerais
3,Rio de Janeiro,/wiki/Rio_de_Janeiro_(estado)


# Looping

In [4]:
links_visitados =[]

def extrai_vizinhos(origem, nome_origem):
    
    if sum([origem == x for x in links_visitados]) <1: #Evitar ler link repetidas vezes
        links_visitados.append(origem)
        trechos = BeautifulSoup(requests.get(origem).text,'lxml').find('table').find(title="Fronteira")

        i =0
        texto = []
        hrefs = []

        while i == 0:
            if trechos.find_next('a').text not in ["População","Regiões geográficas intermediárias"]:
                trechos = trechos.find_next('a')
                texto.append(trechos.text)
                hrefs.append(trechos.get('href'))
            else:
                i=1

        df = pd.DataFrame([texto, hrefs] ).T
        df.columns = ['Estado_Vizinho', 'link_wikipedia']
        df['origem'] = nome_origem

        radical = "https://pt.wikipedia.org/"
        df['link_wiki_full'] = radical + df['link_wikipedia']

        return(df)

In [5]:
#Inicializo com um ponto conhecido
df_sp = extrai_vizinhos('https://pt.wikipedia.org/wiki/Bahia', 'Bahia')

df_sp0 = df_sp[['link_wiki_full','Estado_Vizinho']].drop_duplicates()
for i in range(0, len(df_sp0)):
    df_row = extrai_vizinhos(df_sp0.iloc[i]['link_wiki_full'], df_sp0.iloc[i]['Estado_Vizinho'])
    df_sp=df_sp.append(df_row)

df_sp0= df_sp[['link_wiki_full','Estado_Vizinho']].drop_duplicates()
for i in range(0, len(df_sp0)):
    df_row = extrai_vizinhos(df_sp0.iloc[i]['link_wiki_full'], df_sp0.iloc[i]['Estado_Vizinho'])
    df_sp=df_sp.append(df_row)

df_sp

,Estado_Vizinho,link_wikipedia,origem,link_wiki_full
0,Minas Gerais,/wiki/Minas_Gerais,Bahia,https://pt.wikipedia.org//wiki/Minas_Gerais
1,Goiás,/wiki/Goi%C3%A1s,Bahia,https://pt.wikipedia.org//wiki/Goi%C3%A1s
2,Tocantins,/wiki/Tocantins,Bahia,https://pt.wikipedia.org//wiki/Tocantins
3,Piauí,/wiki/Piau%C3%AD,Bahia,https://pt.wikipedia.org//wiki/Piau%C3%AD
4,Pernambuco,/wiki/Pernambuco,Bahia,https://pt.wikipedia.org//wiki/Pernambuco
5,Alagoas,/wiki/Alagoas,Bahia,https://pt.wikipedia.org//wiki/Alagoas
6,Sergipe,/wiki/Sergipe,Bahia,https://pt.wikipedia.org//wiki/Sergipe
7,Espírito Santo,/wiki/Esp%C3%ADrito_Santo_(estado),Bahia,https://pt.wikipedia.org//wiki/Esp%C3%ADrito_S...
0,São Paulo,/wiki/S%C3%A3o_Paulo_(estado),Minas Gerais,https://pt.wikipedia.org//wiki/S%C3%A3o_Paulo_...
1,Mato Grosso do Sul,/wiki/Mato_Grosso_do_Sul,Minas Gerais,https://pt.wikipedia.org//wiki/Mato_Grosso_do_Sul


In [6]:
links_visitados

['https://pt.wikipedia.org/wiki/Bahia',
 'https://pt.wikipedia.org//wiki/Minas_Gerais',
 'https://pt.wikipedia.org//wiki/Goi%C3%A1s',
 'https://pt.wikipedia.org//wiki/Tocantins',
 'https://pt.wikipedia.org//wiki/Piau%C3%AD',
 'https://pt.wikipedia.org//wiki/Pernambuco',
 'https://pt.wikipedia.org//wiki/Alagoas',
 'https://pt.wikipedia.org//wiki/Sergipe',
 'https://pt.wikipedia.org//wiki/Esp%C3%ADrito_Santo_(estado)',
 'https://pt.wikipedia.org//wiki/S%C3%A3o_Paulo_(estado)',
 'https://pt.wikipedia.org//wiki/Mato_Grosso_do_Sul',
 'https://pt.wikipedia.org//wiki/Distrito_Federal_(Brasil)',
 'https://pt.wikipedia.org//wiki/Bahia',
 'https://pt.wikipedia.org//wiki/Rio_de_Janeiro_(estado)',
 'https://pt.wikipedia.org//wiki/Mato_Grosso',
 'https://pt.wikipedia.org//wiki/Par%C3%A1',
 'https://pt.wikipedia.org//wiki/Maranh%C3%A3o',
 'https://pt.wikipedia.org//wiki/Cear%C3%A1',
 'https://pt.wikipedia.org//wiki/Para%C3%ADba']

# Correcao bolivia

In [7]:
# website_url = requests.get('https://pt.wikipedia.org//wiki/Norte').text
# soup = BeautifulSoup(website_url,'lxml')
# # print(soup.prettify())

# My_table = soup.find('table')

# i =0
# links = My_table.find(title="Fronteira")

In [8]:
# links#.find_next('a').find_next('a').find_next('a').find_next('a').find_next('a').find_next('a')